In [247]:
import pandas as pd
import numpy as np
#Import required libraries
import requests
import re
from bs4 import BeautifulSoup
from geopy import Nominatim
import geopy.distance
#import geocoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [3]:
df = pd.read_csv('../data/dataframe_recom_system_NR.csv')

In [6]:
df.columns

Index(['Unnamed: 0', 'InvoiceNo', 'StockCode', 'Description', 'Quantity',
       'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'Day', 'Month',
       'Year', 'WeekNum', 'Rev', 'Return', 'B2B', 'AbsTrans', 'remove'],
      dtype='object')

In [4]:
df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Day,Month,Year,WeekNum,Rev,Return,B2B,AbsTrans,remove
0,0,537626,85232D,SET/3 DECOUPAGE STACKING TINS,3,2010-12-07 14:57:00,4.95,12347.0,Iceland,Tuesday,December,2010,49,14.85,0,1.0,14.85,False
1,1,537626,85167B,BLACK GRAND BAROQUE PHOTO FRAME,30,2010-12-07 14:57:00,1.25,12347.0,Iceland,Tuesday,December,2010,49,37.50,0,1.0,37.50,False
2,2,537626,84997D,PINK 3 PIECE POLKADOT CUTLERY SET,6,2010-12-07 14:57:00,3.75,12347.0,Iceland,Tuesday,December,2010,49,22.50,0,1.0,22.50,False
3,3,537626,84997C,BLUE 3 PIECE POLKADOT CUTLERY SET,6,2010-12-07 14:57:00,3.75,12347.0,Iceland,Tuesday,December,2010,49,22.50,0,1.0,22.50,False
4,4,537626,84997B,RED 3 PIECE RETROSPOT CUTLERY SET,6,2010-12-07 14:57:00,3.75,12347.0,Iceland,Tuesday,December,2010,49,22.50,0,1.0,22.50,False


In [5]:
#Finding Customers with more than 1 Country listed
df_customer_stg = df.groupby('CustomerID')['Country'].unique()
df_customer_stg.loc[df_customer_stg.apply(lambda x:len(x)>1)]

CustomerID
12370.0           [Cyprus, Austria]
12394.0          [Denmark, Belgium]
12417.0            [Belgium, Spain]
12422.0    [Australia, Switzerland]
12429.0          [Denmark, Austria]
12431.0        [Belgium, Australia]
12455.0             [Cyprus, Spain]
12457.0       [Switzerland, Cyprus]
Name: Country, dtype: object

In [146]:
len(df['Country'].unique())

37

In [232]:
cuntries = []
for i in df['Country'].unique():
    cuntries.append(i.upper())

In [233]:
df_count = pd.DataFrame(cuntries)

In [234]:
df_count.head()

,0
0,ICELAND
1,FINLAND
2,ITALY
3,NORWAY
4,BAHRAIN


In [ ]:
Things I want in the clustering file...
'CustomerID'
'Country'
No of items
No of transactions
Avg transaction spend
Min trans
Max trans
Distance to UK

In [275]:
df.head(1)

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Day,Month,Year,WeekNum,Rev,Return,B2B,AbsTrans,remove
0,0,537626,85232D,SET/3 DECOUPAGE STACKING TINS,3,2010-12-07 14:57:00,4.95,12347.0,Iceland,Tuesday,December,2010,49,14.85,0,1.0,14.85,False


In [238]:
df_summary = df.groupby('CustomerID').agg({'Rev': ['sum','count'],'InvoiceNo':'count','Quantity':'sum'})

In [240]:
df_summary.head()

Rev       InvoiceNo Quantity
                sum count     count      sum
CustomerID                                  
12347.0     4310.00   182       182     2458
12348.0     1437.24    27        27     2332
12349.0     1457.55    72        72      630
12350.0      294.40    16        16      196
12352.0     1265.41    70        70      463

In [243]:
df_summary = df_summary.reset_index()
df_summary.head()

CustomerID      Rev       InvoiceNo Quantity
                  sum count     count      sum
0    12347.0  4310.00   182       182     2458
1    12348.0  1437.24    27        27     2332
2    12349.0  1457.55    72        72      630
3    12350.0   294.40    16        16      196
4    12352.0  1265.41    70        70      463

In [244]:
df_summary.columns = ['CustomerID','TotalRev','ItemCount','NoTrans','TotQty']

In [245]:
df_summary.head()

,CustomerID,TotalRev,ItemCount,NoTrans,TotQty
0,12347.0,4310.00,182,182,2458
1,12348.0,1437.24,27,27,2332
2,12349.0,1457.55,72,72,630
3,12350.0,294.40,16,16,196
4,12352.0,1265.41,70,70,463



# Building the model 1
Now that we have all features ready to go, we will build the model based on those to see which is the output. Initially will work with 5 clusters, expecting, hypothetically, the following:

One cluster for residential stations
One cluster for office stations
One cluster for stations located in elevated points of the city
One cluster for very central/hybrid stations (that have bikes in and out constantly)
One cluster for seaside stations

In [ ]:
# df_summary = oct_features_gps

In [250]:
kmeans = KMeans(n_clusters=5)
october_clusters = kmeans.fit(df_summary) #XXXXXXXX
october_clusters.cluster_centers_

array([[1.53154861e+04, 7.96712025e+02, 5.64333742e+01, 5.64333742e+01,
        4.95857914e+02],
       [1.46460000e+04, 2.79138020e+05, 2.06400000e+03, 2.06400000e+03,
        1.97420000e+05],
       [1.51853125e+04, 3.95318369e+04, 8.07750000e+02, 8.07750000e+02,
        2.70945625e+04],
       [1.50201468e+04, 6.89804729e+03, 2.79137615e+02, 2.79137615e+02,
        4.51429817e+03],
       [1.81020000e+04, 2.59657300e+05, 4.31000000e+02, 4.31000000e+02,
        6.41240000e+04]])


# Building the model 2: With PCA
As we are having more than 25 features, it might make sense to reduce the number of components into 5 and compare with the result of the model 1.

In [251]:
# Principal Component 
pca = PCA(n_components=5)

principalComponents = pca.fit_transform(df_summary)
principalDf = pd.DataFrame(data = principalComponents
             ,columns = ['pc1', 'pc2', 'pc3', 'pc4','pc5'])
principalDf.head()

,pc1,pc2,pc3,pc4,pc5
0,3353.580570,-2936.014157,220.076406,120.969703,-5.024920e-14
1,774.369319,-3031.262847,-1049.739633,-86.011587,6.003462e-14
2,-28.516416,-2927.647012,445.901321,8.666208,1.499042e-12
3,-1257.536627,-2942.693311,266.157964,-58.580940,1.836935e-15
4,-277.421551,-2921.598086,499.123915,9.549190,-1.436881e-15


In [252]:
kmeans = KMeans(n_clusters=5)
october_clusters_PCA = kmeans.fit(principalDf)
october_clusters_PCA.cluster_centers_

array([[-6.83959327e+02,  1.66935919e+01,  3.79858447e+01,
        -1.22259519e+01,  5.68000508e-16],
       [ 2.56722288e+05,  8.33561424e+03,  6.88690003e+04,
        -1.53943494e+03, -2.86411295e-14],
       [ 4.60917291e+04, -3.07092706e+02, -4.55700964e+03,
         4.30856654e+02,  1.82707554e-14],
       [ 6.54080017e+03, -3.10789887e+02, -5.07236605e+02,
         2.07875043e+02, -3.98989742e-15],
       [ 3.38150829e+05, -2.38638875e+03, -3.79906935e+04,
        -1.73068096e+03,  9.71632166e-15]])

# Adding the models to our dataframe


In [253]:
df_summary['labels'] = october_clusters.fit_predict(df_summary)
df_summary.head()

,CustomerID,TotalRev,ItemCount,NoTrans,TotQty,labels
0,12347.0,4310.00,182,182,2458,0
1,12348.0,1437.24,27,27,2332,2
2,12349.0,1457.55,72,72,630,2
3,12350.0,294.40,16,16,196,2
4,12352.0,1265.41,70,70,463,2


In [254]:
df_summary['labelsPCA'] = october_clusters_PCA.fit_predict(principalDf)
df_summary.head()

,CustomerID,TotalRev,ItemCount,NoTrans,TotQty,labels,labelsPCA
0,12347.0,4310.00,182,182,2458,0,0
1,12348.0,1437.24,27,27,2332,2,3
2,12349.0,1457.55,72,72,630,2,3
3,12350.0,294.40,16,16,196,2,3
4,12352.0,1265.41,70,70,463,2,3


In [255]:
df_summary[['labels','labelsPCA']] = df_summary[['labels','labelsPCA']].replace([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

In [256]:
df_summary.labels.value_counts()

3    4071
1     222
4      16
2       1
5       1
Name: labels, dtype: int64

In [257]:
df_summary.labelsPCA.value_counts()

4    4071
1     222
3      16
2       1
5       1
Name: labelsPCA, dtype: int64

In [258]:
#Let's check if there is any relationship between the labels created by the two models:
df_summary.groupby(["labels","labelsPCA"]).count()

,,CustomerID,TotalRev,ItemCount,NoTrans,TotQty
labels,labelsPCA,,,,,
1,1,222,222,222,222,222
2,2,1,1,1,1,1
3,4,4071,4071,4071,4071,4071
4,3,16,16,16,16,16
5,5,1,1,1,1,1


In [260]:
print(f'Out of {df_summary.shape[0]} stations with its labels, only 2 of them have labels that do not match together.')
#Although the models have numbered the labels with different numbers we consider them equivalent

Out of 4311 stations with its labels, only 2 of them have labels that do not match together.


In [ ]:
df_summary.to_csv("..\\Dataset\\clusters.csv",encoding="utf_8",decimal=',', sep=';', index=True)

# adding the countries

In [274]:
df_count.head(1)

,Country,Longitude,Latitude
0,ICELAND,-18.105901,64.984182


In [266]:
df_count.columns = ['Country']

In [267]:
df_count.head()

,Country
0,ICELAND
1,FINLAND
2,ITALY
3,NORWAY
4,BAHRAIN


In [268]:
df_count['Longitude'] = df_count['Country'].apply(geolocator.geocode).apply(lambda x: x.longitude)
df_count['Latitude'] = df_count['Country'].apply(geolocator.geocode).apply(lambda x: x.latitude)

In [269]:
df_count.head()

,Country,Longitude,Latitude
0,ICELAND,-18.105901,64.984182
1,FINLAND,25.920916,63.246778
2,ITALY,12.674297,42.638426
3,NORWAY,9.099972,60.500021
4,BAHRAIN,50.534461,26.155125


In [270]:
df_count.isna().sum()

Country      0
Longitude    0
Latitude     0
dtype: int64

In [272]:
df_count.describe()

,Longitude,Latitude
count,37.000000,37.000000
mean,12.899791,38.909331
std,54.726880,22.606067
min,-145.799667,-28.816624
25%,-3.276575,33.875063
50%,13.199959,42.638426
75%,33.145128,52.865196
max,139.239418,64.984182


In [281]:
df_count[df_count['Country'] == 'UNITED KINGDOM']

,Country,Longitude,Latitude
25,UNITED KINGDOM,-3.276575,54.702354


In [ ]:
#Calculating the distance of each station to the city centre
#We will use London, Lat: 41.3870154, Long: 2.1700471 as the centre of the city and will caluclate the distance, in kilometres, of the station to the centre.

#51.36N
#00.05W

In [288]:
UK = (54.702354, -3.276575)

In [289]:
#Iterating between stations to find the distance from Plaça Ctalunya
stations = range(df_count.shape[0])
distances = []
for s in stations:
    location = (df_count.iloc[s]['Latitude'],df_count.iloc[s]['Longitude'])
    distances.append(round(geopy.distance.vincenty(UK, location).km,2))

C:\Users\ellio\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  


In [290]:
#Adding distances as feature
df_count['UKDist'] = distances   



In [293]:
df_count.sort_values(by='UKDist')

,Country,Longitude,Latitude,UKDist
25,UNITED KINGDOM,-3.276575,54.702354,0.00
31,EIRE,-7.979460,52.865196,371.24
27,NETHERLANDS,5.748082,52.500170,645.17
10,BELGIUM,4.666715,50.640281,701.44
12,DENMARK,10.333328,55.670249,872.15
14,FRANCE,1.888334,46.603354,971.52
3,NORWAY,9.099972,60.500021,979.29
15,GERMANY,10.423447,51.083420,1004.01
7,SWITZERLAND,8.231974,46.798562,1194.00
18,SWEDEN,14.520858,59.674971,1205.04
